In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
sys.path.append("/Users/devenshidfar/Desktop/Masters/NRSC_510B/cebra_control_recal")
import old_jupyter_notebook_version.cebra_utils as cebra_utils
from old_jupyter_notebook_version.cebra_utils import *
import cebra
importlib.reload(cebra_utils)
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap
import pickle  # For saving filtered expt


In [ ]:
# Load data
file_path = os.path.join('/Users/devenshidfar/Desktop/Masters/NRSC_510B/'
                         'cebra_control_recal/mat_code_and_data/',
                         'data/NN_opticflow_dataset.mat')
data = scipy.io.loadmat(file_path, squeeze_me=True, struct_as_record=False)
expt_optic_flow = data['expt']

file_path = os.path.join('/Users/devenshidfar/Desktop/Masters/NRSC_510B/',
                         'cebra_control_recal/mat_code_and_data/'
                         'data/expt_landmark.mat')
data = scipy.io.loadmat(file_path, squeeze_me=True, struct_as_record=False)
expt_landmark = data['expt']

print(expt_optic_flow.shape)

print(f"Experiment data type: {expt_optic_flow.dtype}")
print(f"Experiment data shape: {expt_optic_flow.shape}")

print(expt_landmark.shape)

print(f"Experiment data type: {expt_landmark.dtype}")
print(f"Experiment data shape: {expt_landmark.shape}")

In [ ]:
# THINGS TO DO:
# take a fourier transform of the H decode to find strongest frequencies
#fix the avg angle diff calculation
#normalize cebra data to be on unit sphere/see if then it lives on the unit sphere
#plot scatter of SI score vs num_clusters used

# Config Parameters
session_choose = True

if(session_choose == True):
    landmark_sessions = []
    optic_flow_sessions = []
else:
    landmark_num_trials = 20
    landmark_control_point = 5

    optic_flow_num_trials = 14
    optic_flow_control_point = 14



max_num_reruns = 1

expts = [
    ("optic_flow", expt_optic_flow, optic_flow_control_point, optic_flow_num_trials),
    ("landmark", expt_landmark, landmark_control_point, landmark_num_trials)
]

save_folder = 'neural_save'

for expt_name, expt, control_point, num_trials in expts:


    all_sessions_data = []
    all_neural_data = []
    all_embeddings_3d = []
    all_principal_curves_3d = []
    all_curve_params_3d = []
    all_binned_hipp_angle = []
    all_binned_true_angle = []
    all_binned_est_gain = []
    all_binned_high_vel = []
    all_decoded_angles = []
    all_filtered_decoded_angles_unwrap = []
    all_decode_H = []
    all_session_idx = []
    all_rat = []
    all_day = []
    all_epoch = []
    all_num_skipped_clusters = []
    all_num_used_clusters = []
    all_avg_skipped_cluster_isolation_quality = []
    all_avg_used_cluster_isolation_quality = []
    all_mean_distance_to_principal_curve = []
    all_mean_angle_difference = []
    all_shuffled_mean_angle_difference = []
    all_SI_score_hipp = []
    all_SI_score_true = []
    all_SI_score_high_dim = []
    all_mse_decode_vs_true = []
    all_mean_H_difference = []
    all_std_H_difference = []

    control_count = 0

    model_save_path = os.path.join('/Users/devenshidfar/Desktop/Masters/'
                                   'NRSC_510B/cebra_control_recal/models')

    # Set to 1 to save the models and animations
    save_models = 1
    save_anim = 1
    load_npy = 0
    rm_outliers = True

    # Define bin sizes (in seconds)
    bin_sizes = [1]  # will loop through how ever many bin sizes there are

    # Initialize lists to store embeddings and metadata
    embeddings_list = []
    metadata_list = []

    # Define velocity threshold
    vel_threshold = 5  # degrees per second

    # Main Processing Loop
    for bin_size in bin_sizes:
        print(f"\nProcessing bin_size: {bin_size} second(s)")

        csv_dict_all_data = []

        for session_idx, session in enumerate(expt):
            control_count += 1


            # Control Session Skipping
            if control_count <= control_point:
                print(f"Skipping session {session_idx + 1}")
                continue 
            elif control_count > control_point + num_trials:
                print("Reached the desired number of trials." 
                      "Exiting session loop.")
                break

            print(f"\nProcessing session {session_idx + 1}/{len(expt)}")
            print(
                  f"Rat: {session.rat}, "
                  "Day: {session.day}, "
                  "Epoch: {session.epoch}"
            )

    
            ros_data = session.rosdata

            start_time = ros_data.startTs
            end_time = ros_data.stopTs

            # Convert to seconds (originally in milliseconds)
            enc_times = np.array(ros_data.encTimes - start_time) / 1e6  
            vel = np.array(ros_data.vel)

            # Ensure enc_times and vel are valid
            valid_idx = np.isfinite(enc_times) & np.isfinite(vel)
            enc_times = enc_times[valid_idx]
            vel = vel[valid_idx]

            # Get indices where vel > vel_threshold
            high_vel_idx = vel > vel_threshold

            if np.sum(high_vel_idx) == 0:
                print("No data points where vel > vel_threshold. "
                      "Skipping session.")
                continue

            # Filter enc_times and behavioral variables
            enc_times_high_vel = enc_times[high_vel_idx]
            high_vel_filtered = vel[high_vel_idx]
            est_gain_filtered = ( 
                np.array(ros_data.estGain)[valid_idx][high_vel_idx]
            )
            print(f"est_gain filtered: {est_gain_filtered[100:130]}")
            hipp_angle_filtered = (
                np.array(ros_data.hippAngle)[valid_idx][high_vel_idx]
                )
            true_angle_filtered = (
                np.array(ros_data.encAngle)[valid_idx][high_vel_idx]
                )
            rel_angle_filtered = (
                np.array(ros_data.relAngle)[valid_idx][high_vel_idx]
                )

            # Define bins over enc_times_high_vel (Non-Overlapping)
            bins = np.arange(
                enc_times_high_vel[0], 
                enc_times_high_vel[-1] + bin_size, bin_size
            )

            if len(bins) < 2:
                print("Not enough data after filtering for high velocity. "
                      "Skipping session.")
                continue

            # Bin behavioral variables using non-overlapping bins
            binned_est_gain, _, _ = stats.binned_statistic(
                enc_times_high_vel, 
                est_gain_filtered, 
                statistic='mean', 
                bins=bins
            )
            binned_hipp_angle, _, _ = stats.binned_statistic(
                enc_times_high_vel, 
                hipp_angle_filtered, 
                statistic='mean', 
                bins=bins
            )
            binned_true_angle, _, _ = stats.binned_statistic(
                enc_times_high_vel, 
                true_angle_filtered, 
                statistic='mean', 
                bins=bins
            )
            binned_high_vel, _, _ = stats.binned_statistic(
                enc_times_high_vel, 
                high_vel_filtered, 
                statistic='mean', 
                bins=bins
            )
            binned_rel_angle, _, _ = stats.binned_statistic(
                enc_times_high_vel, 
                rel_angle_filtered, 
                statistic='mean', 
                bins=bins
            )

            # Remove NaN bins
            valid_bins = (
                ~np.isnan(binned_hipp_angle) 
                & ~np.isnan(binned_true_angle) 
                & ~np.isnan(binned_est_gain) 
                & ~np.isnan(binned_high_vel)
            )
            if not np.all(valid_bins):
                print(f"Removing {np.sum(~valid_bins)} bins with NaNs.")
                binned_hipp_angle = binned_hipp_angle[valid_bins]
                binned_true_angle = binned_true_angle[valid_bins]
                binned_est_gain = binned_est_gain[valid_bins]
                binned_high_vel = binned_high_vel[valid_bins]
                binned_rel_angle = binned_rel_angle[valid_bins]
                # Adjust bins to match the number of valid bins
                bins = bins[:-1][valid_bins]  

            # Filter Spike Times Based on High Velocity
            all_spikes = []
            skipped_clusters = 0
            used_clusters = 0
            used_cluster_isolation_quality_list = []
            skipped_cluster_isolation_quality_list = []

            for cluster in session.clust:
                if cluster.isolationQuality > 4:
                    # done in Madhav et al., 2024
                    print(f"Skipping cluster {cluster.name}"
                    "due to low isolation quality ({cluster.isolationQuality})") 
                    skipped_clusters += 1
                    skipped_cluster_isolation_quality_list.append(cluster.isolationQuality)
                    continue
                else:
                    used_clusters += 1
                    used_cluster_isolation_quality_list.append(cluster.isolationQuality)

                # Convert to seconds
                spike_times_sec = (cluster.ts - start_time) / 1e6  
                vel_at_spikes = cluster.vel

                # Include spikes where vel_at_spikes > vel_threshold
                include_spikes = vel_at_spikes > vel_threshold
                spike_times_sec_high_vel = spike_times_sec[include_spikes]

                if len(spike_times_sec_high_vel) == 0:
                    print(f"No spikes for cluster {cluster.name}" 
                        "at high velocities. Skipping cluster.")
                    continue

                # Bin spikes using non-overlapping bins
                binned_spikes, _, _ = stats.binned_statistic(
                    spike_times_sec_high_vel,
                    np.ones_like(spike_times_sec_high_vel),
                    statistic='sum',
                    bins=bins
                )
                # Note: In non-overlapping bins, 
                # we don't assign NaNs as in overlapping bins

                # Append to all_spikes
                all_spikes.append(binned_spikes)

            #Cluster statistics

            used_cluster_isolation_quality = np.asarray(used_cluster_isolation_quality_list)
            skipped_cluster_isolation_quality = np.asarray(skipped_cluster_isolation_quality_list)

            num_skipped_cluster = len(skipped_cluster_isolation_quality)
            num_used_cluster = len(used_cluster_isolation_quality)

            avg_skipped_cluster_isolation_quality = np.mean(skipped_cluster_isolation_quality)
            avg_used_cluster_isolation_quality = np.mean(used_cluster_isolation_quality)

            if not all_spikes:
                print("No valid spike data after filtering. Skipping session.")
                continue

            # Assemble neural data: shape (num_bins, num_clusters)
            neural_data = np.array(all_spikes).T  # Shape: (num_bins, num_clusters)

            # Consistency Check Between Bins
            num_bins_neural = neural_data.shape[0]
            num_bins_behavior = len(binned_est_gain)

            if num_bins_neural != num_bins_behavior:
                print(f"Warning: Number of neural data bins ({num_bins_neural})" 
                    "does not match behavioral data bins ({num_bins_behavior})." 
                    "Adjusting to minimum.")
                min_bins = min(num_bins_neural, num_bins_behavior)
                neural_data = neural_data[:min_bins, :]
                binned_est_gain = binned_est_gain[:min_bins]
                binned_hipp_angle = binned_hipp_angle[:min_bins]
                binned_true_angle = binned_true_angle[:min_bins]
                binned_high_vel = binned_high_vel[:min_bins]
                bins = bins[:min_bins]

            binned_est_gain_temp = binned_est_gain
            binned_hipp_angle_temp = binned_hipp_angle
            binned_true_angle_temp = binned_true_angle
            binned_high_vel_temp = binned_high_vel

            embeddings_2d_list = []
            embeddings_3d_list = []

            embedding_filename_2d = f"bin_size-{bin_size}_embeddings_2d_rat"
            "{session.rat}_day{session.day}_epoch{session.epoch}.npy"
            embedding_filename_3d = f"bin_size-{bin_size}_embeddings_3d_rat"
            "{session.rat}_day{session.day}_epoch{session.epoch}.npy"


            compile_path = f'/Users/devenshidfar/Desktop/Masters/NRSC_510B/'
            'cebra_control_recal/results/{expt_name}/{save_folder}'
            os.makedirs(compile_path,exist_ok=True)

            temperature = [1]  
            for temp in temperature:
    
                binned_est_gain = binned_est_gain_temp
                binned_hipp_angle = binned_hipp_angle_temp
                binned_true_angle = binned_true_angle_temp
                binned_high_vel = binned_high_vel_temp


                base_path = f'/Users/devenshidfar/Desktop/Masters/NRSC_510B/'
                'cebra_control_recal/results/{save_folder}/{expt_name}/'
                'temperature_{temp}'
                results_save_path = f'{base_path}/rat_{session.rat}/'
                'session_{session_idx}'
                anim_save_path = f'{results_save_path}/3d_animations'
                time_dist_save_path = f'{results_save_path}/time_dist_plot'
                SI_save_path = f'{results_save_path}/SI_Plots'
                param_plot_path = f'{results_save_path}/Param_Plots'
                behav_var_plot_path = f'{results_save_path}/behav_var_plot'
                csv_save_path = f'{results_save_path}/csv_file'
                spectrogram_path = f'{results_save_path}/spatial_spectrograms'

                os.makedirs(results_save_path,exist_ok=True)

                pdf_filename = os.path.join(results_save_path, f'session_{session_idx}.pdf')
                pdf = PdfPages(pdf_filename)

                # Create necessary directories
                os.makedirs(model_save_path, exist_ok=True)
                os.makedirs(anim_save_path, exist_ok=True)
                os.makedirs(SI_save_path, exist_ok=True)
                os.makedirs(param_plot_path, exist_ok=True)
                os.makedirs(csv_save_path, exist_ok=True)
                os.makedirs(spectrogram_path, exist_ok=True)

                # Apply CEBRA embeddings
                # embeddings_2d = apply_cebra(neural_data, 2,temperature=temp)
                # embeddings_3d = apply_cebra(neural_data, 3,temperature=temp)

                num_reruns = 0
                for num_reruns in range(max_num_reruns):

                    def apply_cebra(neural_data=None,output_dimension=3,temperature=1):

                        ''' default hyper-params for CEBRA model

                        # Model Architecture (model_architecture): 'offset10-model'
                        # Batch Size (batch_size): 512
                        # Temperature Mode (temperature_mode): "auto"
                        # Learning Rate (learning_rate): 0.001
                        # Max Iterations (max_iterations): 10,000
                        # Time Offsets (time_offsets): 10
                        # Output Dimension (output_dimension): 8
                        # Device (device): "cuda_if_available" (falls back to "cpu" if no GPU is available)
                        # Verbose (verbose): False

                        '''
                        
                        model = cebra.CEBRA(
                            output_dimension=output_dimension, 
                            max_iterations=1000, 
                            batch_size = 512,
                            temperature=temperature,)   
                        model.fit(neural_data)
                        embeddings = model.transform(neural_data)
                        return embeddings
                    
                    # Shape: (num_bins, num_clusters)
                    neural_data = np.array(all_spikes).T 
                    embeddings_high_dim = apply_cebra(neural_data, 3, temperature=temp)

                    # from sklearn.manifold import Isomap

                    # isomap = Isomap(n_components=3)
                    # embeddings_3d = isomap.fit_transform(embeddings_high_dim)

                    embeddings_3d = embeddings_high_dim

                    print(f"Output embeddings_3d shape before nt_TDA and mask: {embeddings_3d.shape}")

                    # Create NaN mask based on behavioral variables and embeddings
                    nan_mask_3d = (
                        ~np.isnan(embeddings_3d).any(axis=1) &
                        ~np.isnan(binned_hipp_angle) &
                        ~np.isnan(binned_true_angle) &
                        ~np.isnan(binned_est_gain) &
                        ~np.isnan(binned_high_vel)
                    )

                    # embeddings_2d = embeddings_2d[nan_mask_2d, :]
                    embeddings_3d = embeddings_3d[nan_mask_3d, :]
                    binned_hipp_angle = binned_hipp_angle[nan_mask_3d]
                    binned_true_angle = binned_true_angle[nan_mask_3d]
                    binned_est_gain = binned_est_gain[nan_mask_3d]
                    binned_high_vel = binned_high_vel[nan_mask_3d]

                    nan_mask_high_dim = (
                        ~np.isnan(embeddings_high_dim).any(axis=1) &
                        ~np.isnan(binned_hipp_angle) &
                        ~np.isnan(binned_true_angle) &
                        ~np.isnan(binned_est_gain) &
                        ~np.isnan(binned_high_vel)
                    )

                    #for high dim
                    embeddings_high_dim = embeddings_high_dim[nan_mask_high_dim, :]
                    binned_hipp_angle_high_dim = binned_hipp_angle[nan_mask_high_dim]

                    # print(f"Output embeddings_2d shape after NaN mask: {embeddings_2d.shape}")
                    print(f"Output embeddings_3d shape after NaN mask: {embeddings_3d.shape}")

                    if rm_outliers:
                        print("Removing outliers...")
                        #inlier_indices_2d = nt_TDA(embeddings_2d)
                        inlier_indices_3d = nt_TDA(embeddings_3d)
                        inlier_indices_high_dim = nt_TDA(embeddings_high_dim)
                        #embeddings_2d = embeddings_2d[inlier_indices_2d, :]
                        embeddings_3d = embeddings_3d[inlier_indices_3d, :]
                        binned_hipp_angle = binned_hipp_angle[inlier_indices_3d]
                        binned_true_angle = binned_true_angle[inlier_indices_3d]
                        binned_est_gain = binned_est_gain[inlier_indices_3d]
                        binned_high_vel = binned_high_vel[inlier_indices_3d]
                        #high dim
                        embeddings_high_dim = embeddings_high_dim[inlier_indices_high_dim, :]
                        binned_hipp_angle_high_dim = binned_hipp_angle_high_dim[inlier_indices_high_dim]

                    #print(f"Output embeddings_2d shape after outlier removal: {embeddings_2d.shape}")
                    print(f"Output embeddings_3d shape after outlier removal: "
                            "{embeddings_3d.shape}")

                    print(f"binned_est_gain: {binned_est_gain[100:130]}")

                    # Convert angles to radians
                    binned_true_angle_rad = np.deg2rad(binned_true_angle)
                    binned_hipp_angle_rad = np.deg2rad(binned_hipp_angle)
                    binned_rel_angle_rad = np.deg2rad(binned_rel_angle)
                    binned_high_vel_rad = np.deg2rad(binned_high_vel)
                    binned_hipp_angle_rad_high_dim = np.deg2rad(binned_hipp_angle_high_dim)

                    print(f'binned true angle: {binned_true_angle_rad[:30]}')
                    print(f'binned hipp angle: {binned_hipp_angle_rad[:30]}')
                    print(f'binned rel angle: {binned_rel_angle_rad[:30]}')
                    print(f'binned high vel: {binned_high_vel_rad[:30]}')

                    binned_true_angle_rad_unwrap = binned_true_angle_rad
                    binned_hipp_angle_rad_unwrap = binned_hipp_angle_rad
                    binned_rel_angle_rad_unwrap = binned_rel_angle_rad
                    binned_hipp_angle_high_dim_unwrap = binned_hipp_angle_high_dim

                    #wrap angles within [0, 2π]
                    binned_true_angle_rad = (binned_true_angle_rad_unwrap 
                                             % (2 * np.pi))
                    binned_hipp_angle_rad = (binned_hipp_angle_rad_unwrap 
                                             % (2 * np.pi))
                    binned_rel_angle_rad = binned_rel_angle % (2 * np.pi)

                    binned_hipp_angle_rad_high_dim = (binned_hipp_angle_high_dim_unwrap 
                                                      % (2 * np.pi))

                    print(f"binned_hipp_angle_rad: {binned_hipp_angle_rad[200:230]}")
                    print(f"binned_true_angle_rad: {binned_true_angle_rad[200:230]}")

                    plot_and_save_behav_vars(
                        binned_hipp_angle=binned_hipp_angle_rad, 
                        binned_true_angle=binned_true_angle_rad, 
                        binned_est_gain=binned_est_gain,
                        save_dir=behav_var_plot_path, 
                        session_idx=session_idx
                    )

                    # plot_in_2d(
                    #     embeddings=embeddings_2d,
                    #     session=session, 
                    #     behav_var=binned_hipp_angle_rad, 
                    #     name_behav_var="Hipp_Angle",
                    #     principal_curve=None
                    # )

                    # Compute SI
                    SI_params = {
                        'n_bins': 10,
                        'n_neighbors': 15,
                        'discrete_label': False,
                        'num_shuffles': 10,
                        'verbose': False,
                    }

                    print(f"length of embeddings_3d: {embeddings_3d.shape[0]}")

                    SI_score_hipp = compute_SI_and_plot(
                        embeddings=embeddings_3d,
                        behav_var=binned_hipp_angle_rad,
                        params=SI_params,
                        behav_var_name='Hipp_Angle',
                        save_dir=SI_save_path,
                        session_idx=session_idx,
                        dimensions_3=True,
                        pdf=pdf,
                        num_used_clusters=num_used_cluster
                    )

                    if(SI_score_hipp >= 0.8):
                        break
                    elif(SI_score_hipp < 0.80):
                        print(f"SI_score_hipp is {SI_score_hipp}."
                              " Retrying another embedding")
                        num_reruns += 1
                        continue

                SI_score_true = compute_SI_and_plot(
                    embeddings=embeddings_3d,
                    behav_var=binned_true_angle_rad,
                    params=SI_params,
                    behav_var_name='True_Angle',
                    save_dir=SI_save_path,
                    session_idx=session_idx,
                    dimensions_3=True,
                    pdf=pdf,
                    num_used_clusters=num_used_cluster
                )

                penalty_types = ['curvature']
                curvature_coeffs = [5]

                for penalty_type in penalty_types:
                    for curv in curvature_coeffs:
                        fit_params = {
                            'dalpha': 0.005,
                            'knot_order': 'nearest',
                            'penalty_type': penalty_type,  # curvature penalty
                            'nKnots': 20,
                            'curvature_coeff': curv,
                            'len_coeff': 2,
                            'density_coeff': 2,
                            'delta': 0.1
                        }

                        principal_curve_3d, principal_curve_3d_pre, curve_params_3d = fit_spud_to_cebra(
                            embeddings=embeddings_3d,
                            ref_angle=binned_true_angle_rad_unwrap,  # Adjust as needed
                            session_idx=session_idx,
                            session=session,
                            results_save_path=results_save_path,
                            fit_params=fit_params,
                            dimension_3d=1,
                            verbose=False
                        )

                        # Create Rotating 3D Plots
                        anim_save_file = f"{anim_save_path}/bin_size-{bin_size}/"
                        "vel_thresh_{vel_threshold}/"
                        "curv_{curv}_pen_type_{penalty_type}/"
                        os.makedirs(anim_save_file, exist_ok=True)


                        # Distance and Decoding Calculations
                        mean_dist_to_princ = dist_tot_to_princ_curve(
                                                                    embeddings=embeddings_3d, 
                                                                    principal_curve=principal_curve_3d)
                        shuffled_hipp_angle_binned_3d = np.random.permutation(binned_hipp_angle_rad)

                        decoded_angles, mse_decode_vs_true = calculate_average_difference_in_decoded_hipp_angle(
                            embeddings=embeddings_3d,
                            principal_curve=principal_curve_3d,
                            tt=curve_params_3d,
                            behav_angles=binned_hipp_angle_rad,
                            true_angles=binned_true_angle_rad
                        )
                        shuffled_decoded_angles, shuffled_mse_decode_vs_true = calculate_average_difference_in_decoded_hipp_angle(
                            embeddings=embeddings_3d,
                            principal_curve=principal_curve_3d,
                            tt=curve_params_3d,
                            behav_angles=shuffled_hipp_angle_binned_3d,
                            true_angles=binned_true_angle_rad 
                        )

                
                        decoded_angles_unwrap = (
                            decoded_angles_unwrap + binned_true_angle_rad_unwrap[3]
                        )

                        shuffled_decoded_angles_unwrap = (
                            shuffled_decoded_angles_unwrap + binned_true_angle_rad_unwrap[3]
                        )

                        decoded_angles = (
                            (decoded_angles + binned_true_angle_rad_unwrap[3]) % (2 * np.pi)
                        )

                        shuffled_decoded_angles = (
                            (shuffled_decoded_angles + binned_true_angle_rad_unwrap[3]) % (2 * np.pi)
                        )


                        print("DECODED_ANGLES")
                        print(decoded_angles_unwrap[:30])

                        print('post decodded angles')

                        print(decoded_angles_unwrap[:30])


                        angle_diff = ((decoded_angles - binned_hipp_angle_rad) 
                                      % (2 * np.pi))

                        shuffled_angle_diff = ((shuffled_decoded_angles - binned_hipp_angle_rad) 
                                               % (2*np.pi))

                        print(angle_diff[:30])
                        mean_angle_diff = np.mean(angle_diff)
                        shuffled_mean_angle_diff = np.mean(shuffled_angle_diff)

                        # # Calculate average angular difference
                        # avg_diff = calculate_average_angle_difference(decoded_angles, actual_angles)

                        # Apply low-pass filter
                        filtered_decoded_angles_unwrap = low_pass_filter(
                            angles=decoded_angles_unwrap,
                            cutoff_frequency=0.2,
                            filter_order=3,
                            fs=1
                        )

                        filtered_decoded_angles_unwrap = savgol_filter(
                            filtered_decoded_angles_unwrap,
                            window_length=30,
                            polyorder=2
                        )

                        derivative_decoded_angle_rad_unwrap = window_smooth(
                            data=filtered_decoded_angles_unwrap,
                            window_size=60
                        )
                        derivative_true_angle_rad_unwrap = window_smooth(
                            data=binned_true_angle_rad_unwrap,
                            window_size=60
                        )
                        derivative_hipp_angle_rad_unwrap = window_smooth(
                            data=binned_hipp_angle_rad_unwrap,
                            window_size=60
                        )

                
                        # Plotting the results
                        plt.figure(figsize=(15, 6))
                        plt.plot(decoded_angles_unwrap, label='Original Decoded Angles', alpha=0.5)
                        plt.plot(shuffled_decoded_angles_unwrap,label='Shuffled Decoded Angles', alpha=0.5)
                        plt.plot(filtered_decoded_angles_unwrap, label='Filtered Decoded Angles', linewidth=2)
                        plt.xlabel('Time (s)')
                        plt.ylabel('Decoded Angle (rad)')
                        plt.title('Low-Pass Filter Applied to Decoded Angles')
                        plt.legend()
                        plt.grid(True)
                        pdf.savefig()
                        plt.show()

                        # Plot Decoded Variables vs true

                        plot_decoded_var_and_true(decoded_var=filtered_decoded_angles_unwrap,
                            behav_var=binned_hipp_angle_rad_unwrap,
                            save_path=param_plot_path,
                            session_idx=session_idx,
                            behav_var_name='Hippocampal Angle', pdf=pdf)
                        
                        #interactive plots

                        # plot_decoded_var_and_true_interactive(
                        #     decoded_var=decoded_angles_unwrap,
                        #     behav_var=binned_hipp_angle_rad_unwrap,
                        #     true_angle=binned_true_angle_rad_unwrap,
                        #     xlabel='Parametrization (0 to 1)',
                        #     ylabel1='Decoded Variable',
                        #     ylabel2='hipp angle',
                        #     legend_labels=['Decoded Variable', 'hipp angle','True Angle'],
                        #     save_path=param_plot_path,
                        #     session_idx=session_idx,
                        #     behav_var_name='Hipp_Angle'
                        # )

                        # plot_decoded_var_and_true_interactive(
                        #     decoded_var=shuffled_decoded_angles_unwrap,
                        #     behav_var=binned_hipp_angle_rad_unwrap,
                        #     true_angle=binned_true_angle_rad_unwrap,
                        #     xlabel='Parametrization (0 to 1)',
                        #     ylabel1='Shuffled Decoded Variable',
                        #     ylabel2='hipp angle',
                        #     legend_labels=['Shuffled Decoded Variable', 'hipp angle','True Angle'],
                        #     save_path=param_plot_path,
                        #     session_idx=session_idx,
                        #     behav_var_name='shuffled_Hipp_Angle'
                        # )

                        # plot_decoded_var_and_true_interactive(
                        #     decoded_var=filtered_decoded_angles_unwrap,
                        #     behav_var=binned_hipp_angle_rad_unwrap,
                        #     true_angle=binned_true_angle_rad_unwrap,
                        #     xlabel='Parametrization (0 to 1)',
                        #     ylabel1='Filtered Decoded Variable',
                        #     ylabel2='Hipp angle',
                        #     legend_labels=['Filtered Decoded Variable', 'Hipp angle','True Angle'],
                        #     save_path=param_plot_path,
                        #     session_idx=session_idx,
                        #     behav_var_name='Hipp_Angle_filtered'
                        # )

                        # plot_decoded_var_and_true_interactive(
                        #     decoded_var=derivative_decoded_angles_unwrap,
                        #     behav_var=derivative_hipp_angle_unwrap,
                        #     true_angle=derivative_true_angle_unwrap,
                        #     xlabel='Parametrization (0 to 1)',
                        #     ylabel1='derivatives of filtered angles',
                        #     ylabel2='Behavioral Variable',
                        #     legend_labels=['Decoded Variable', 'Hipp Angle Derivative','True Angle Derivative'],
                        #     save_path=param_plot_path,
                        #     session_idx=session_idx,
                        #     behav_var_name="derivatives"
                        # )

                        print(f"angle_diff random 10: {angle_diff[20:30]}")
                        compare_angles = np.column_stack((
                            shuffled_hipp_angle_binned_3d,
                            binned_hipp_angle_rad,
                            decoded_angles
                        ))

                        print(f"compare angle random 10: {compare_angles[20:30]}")

                        plot_in_3d_static(
                            embeddings_3d=embeddings_3d,
                            session=session,
                            behav_var=binned_hipp_angle_rad,
                            name_behav_var="Hippocampal_Angle",
                            save_path=anim_save_file,
                            principal_curve=principal_curve_3d,
                            tt=None,
                            num_labels=20,
                            mean_dist=mean_dist_to_princ,
                            avg_angle_diff=mean_angle_diff,
                            shuffled_avg_angle_diff=shuffled_mean_angle_diff,
                            pdf=pdf
                        )

                        plot_in_3d_static(
                            embeddings_3d=embeddings_3d,
                            session=session,
                            behav_var=binned_hipp_angle_rad,
                            name_behav_var="Hippocampal_Angle ",
                            save_path=anim_save_file,
                            tt=None,
                            num_labels=20,
                            mean_dist=mean_dist_to_princ,
                            avg_angle_diff=mean_angle_diff,
                            shuffled_avg_angle_diff=shuffled_mean_angle_diff,
                            pdf=pdf
                        )

                        # Create Rotating 3D Plot
                        create_rotating_3d_plot(
                            embeddings_3d=embeddings_3d,
                            session=session,
                            behav_var=binned_hipp_angle_rad,
                            name_behav_var="hipp angle",
                            anim_save_path=anim_save_file,
                            save_anim=save_anim,
                            principal_curve=principal_curve_3d,
                            tt=curve_params_3d,
                            num_labels=20,
                            mean_dist=mean_dist_to_princ,
                            avg_angle_diff=mean_angle_diff,
                            shuffled_avg_angle_diff=shuffled_mean_angle_diff
                        )
                        print(f"Created rotating 3D plots for session {session_idx}.")

                # Decoding H
                decode_H = (derivative_decoded_angle_rad_unwrap) / (derivative_true_angle_rad_unwrap)

                decode_H_nan_count = np.isnan(decode_H).sum()
                decode_H_nan_indices = np.where(np.isnan(decode_H))[0]

                # For est_gain_binned
                est_gain_nan_count = np.isnan(binned_est_gain).sum()
                est_gain_nan_indices = np.where(np.isnan(binned_est_gain))[0]

                # Print results
                print(f"Mean of decode H (ignoring NaNs): {np.nanmean(decode_H)}")
                print(f"Mean of est gain H (ignoring NaNs): {np.nanmean(binned_est_gain)}")

                print(f"Number of NaNs in decode H: {decode_H_nan_count}")
                print(f"Indices of NaNs in decode H: {decode_H_nan_indices}")

                print(f"Number of NaNs in est gain H: {est_gain_nan_count}")
                print(f"Indices of NaNs in est gain H: {est_gain_nan_indices}")

                decode_H = np.clip(decode_H, -2, 2)

                min_len = min(len(binned_est_gain), len(decode_H))

                mean_H_diff = np.mean(np.abs(binned_est_gain[:min_len] - decode_H[:min_len]))
                std_H_diff = np.std(binned_est_gain[:min_len] - decode_H[:min_len])

                lap_number, sorted_decode_H, sorted_lap_number = get_var_over_lap(
                    var=decode_H,
                    true_angle=binned_true_angle_rad_unwrap
                )

                _, sorted_H_est, _ = get_var_over_lap(
                    var=binned_est_gain,
                    true_angle=binned_true_angle_rad_unwrap
                )

                _, sorted_vel, _ = get_var_over_lap(
                    var=binned_high_vel,
                    true_angle=binned_true_angle_rad_unwrap
                )

                _, sorted_rel_angle, _ = get_var_over_lap(
                    var=binned_rel_angle_rad,
                    true_angle=binned_true_angle_rad_unwrap
                )

                _, sorted_hipp_angle, _ = get_var_over_lap(
                    var=binned_hipp_angle_rad,
                    true_angle=binned_true_angle_rad_unwrap
                )

                # get hipp angle laps for spatial spectrogram
                hipp_lap_number, hipp_decode_H, sorted_hipp_lap_number = get_var_over_lap(
                    var=decode_H,
                    true_angle=binned_hipp_angle_rad_unwrap
                )
                # Plot spatial spectrogram
                # True frame
                plot_spatial_spectrogram(
                    H=sorted_decode_H,
                    lap_numbers=sorted_lap_number,
                    behav_var_name="True_Frame",
                    save_path=spectrogram_path,
                    num_segments_per_lap=10,
                )

                # Hipp frame
                plot_spatial_spectrogram(
                    H=hipp_decode_H,
                    lap_numbers=sorted_hipp_lap_number,
                    behav_var_name="Hipp_Frame",
                    save_path=spectrogram_path,
                    num_segments_per_lap=10,
                )

                # Interactive plots
                plot_spatial_spectrogram_interactive(
                    H=sorted_decode_H,
                    lap_numbers=sorted_lap_number,
                    behav_var_name="True_Frame",
                    save_path=spectrogram_path,
                    session=session,
                    session_idx=session_idx,
                    num_segments_per_lap=10,
                )

                plot_spatial_spectrogram_interactive(
                    H=hipp_decode_H,
                    lap_numbers=sorted_hipp_lap_number,
                    behav_var_name="Hipp_Frame",
                    save_path=spectrogram_path,
                    session=session,
                    session_idx=session_idx,
                    num_segments_per_lap=10,
                )

                print(f"sorted lap number: {np.max(sorted_lap_number) - np.min(sorted_lap_number)}")
                print(f"max lap number: {np.max(sorted_lap_number)}")
                print(f"min lap number: {np.min(sorted_lap_number)}")

                # Plot Hs Over Laps

                sorted_H_est_ma = compute_moving_average(data=sorted_H_est, window_size=20)
                sorted_decode_H_ma = compute_moving_average(data=sorted_decode_H, window_size=20)

            
                # Plot Hs Over Laps with Moving Average
                
                plot_Hs_over_laps_interactive(
                    est_H=sorted_H_est_ma,
                    decode_H=sorted_decode_H_ma,
                    lap_number=sorted_lap_number,
                    behav_var=sorted_vel,
                    session_idx=session_idx,
                    session=session,
                    save_path=results_save_path,
                    tag='vel_ma_20',
                    SI_score=SI_score_hipp,
                    decode_err=mse_decode_vs_true,
                    mean_diff=mean_H_diff,
                    std_diff=std_H_diff,
                    behav_var_name="Velocity"
                )

                plot_Hs_over_laps_interactive(
                    est_H=sorted_H_est,
                    decode_H=sorted_decode_H,
                    lap_number=sorted_lap_number,
                    session_idx=session_idx,
                    session=session,
                    save_path=results_save_path,
                    tag='vel_no_ma',
                    SI_score=SI_score_hipp,
                    decode_err=mse_decode_vs_true,
                    mean_diff=mean_H_diff,
                    std_diff=std_H_diff,
                    behav_var_name="Velocity"
                )
            
                # Plot Hs Over Laps with Moving Average
                
                plot_Hs_over_laps_interactive(
                    est_H=sorted_H_est_ma,
                    decode_H=sorted_decode_H_ma,
                    lap_number=sorted_lap_number,
                    behav_var=sorted_rel_angle,
                    session_idx=session_idx,
                    session=session,
                    save_path=results_save_path,
                    tag='rel_ma_20',
                    SI_score=SI_score_hipp,
                    decode_err=mse_decode_vs_true,
                    mean_diff=mean_H_diff,
                    std_diff=std_H_diff,
                    behav_var_name="Rel_Angle"
                )


                plot_Hs_over_time(
                    est_H=sorted_H_est_ma,
                    decode_H=sorted_decode_H_ma,
                    session_idx=session_idx,
                    behav_var=None,
                    behav_var_name='Rel_Angle',
                    session=session,
                    save_path=results_save_path,
                    tag='rel_ma_20',
                    is_moving_avg=False,
                    SI_score=SI_score_hipp,
                    decode_err=mse_decode_vs_true,
                    pdf=pdf
                )

                binned_est_gain_ma = compute_moving_average(data=binned_est_gain, window_size=20)
                decode_H_ma = compute_moving_average(decode_H, window_size=20)

                #moving avg with vel over time

                plot_Hs_over_time(
                    est_H=binned_est_gain_ma,
                    decode_H=decode_H_ma,
                    session_idx=session_idx,
                    behav_var=None,
                    behav_var_name='Velocity',
                    session=session,
                    save_path=results_save_path,
                    tag='vel_ma_20',
                    is_moving_avg=True,
                    SI_score=SI_score_hipp,
                    decode_err=mse_decode_vs_true
                )
                
                all_neural_data.append(neural_data)
                all_embeddings_3d.append(embeddings_3d)
                all_principal_curves_3d.append(principal_curve_3d)
                all_curve_params_3d.append(curve_params_3d)
                all_binned_hipp_angle.append(binned_hipp_angle_rad_unwrap)
                all_binned_true_angle.append(binned_true_angle_rad_unwrap)
                all_binned_est_gain.append(binned_est_gain)
                all_binned_high_vel.append(binned_high_vel)
                all_decoded_angles.append(decoded_angles_unwrap)
                all_filtered_decoded_angles_unwrap.append(filtered_decoded_angles_unwrap)
                all_decode_H.append(decode_H)
                all_session_idx.append(session_idx)
                all_rat.append(session.rat)
                all_day.append(session.day)
                all_epoch.append(session.epoch)
                all_num_skipped_clusters.append(num_skipped_cluster)
                all_num_used_clusters.append(num_used_cluster)
                all_avg_skipped_cluster_isolation_quality.append(avg_skipped_cluster_isolation_quality)
                all_avg_used_cluster_isolation_quality.append(avg_used_cluster_isolation_quality)
                all_mean_distance_to_principal_curve.append(mean_dist_to_princ)
                all_mean_angle_difference.append(mean_angle_diff)
                all_shuffled_mean_angle_difference.append(shuffled_mean_angle_diff)
                all_SI_score_hipp.append(SI_score_hipp)
                all_SI_score_true.append(SI_score_true)
                all_mse_decode_vs_true.append(mse_decode_vs_true)
                all_mean_H_difference.append(mean_H_diff)
                all_std_H_difference.append(std_H_diff)

                # csv_dict = {
                # 'session_idx': session_idx,
                # 'rat': session.rat,
                # 'day': session.day,
                # 'epoch': session.epoch,
                # 'num_skipped_clusters': num_skipped_cluster,
                # 'num_used_clusters': num_used_cluster,
                # 'avg_skipped_cluster_isolation_quality': avg_skipped_cluster_isolation_quality,
                # 'avg_used_cluster_isolation_quality': avg_used_cluster_isolation_quality,
                # 'mean_distance_to_principal_curve': mean_dist_to_princ,
                # 'mean_angle_difference': mean_angle_diff,
                # 'shuffled_mean_angle_difference': shuffled_mean_angle_diff,
                # 'SI_score_hipp': SI_score_hipp,
                # 'SI_score_true': SI_score_true, 
                # 'SI_score_high_dim': SI_score_high_dim,
                # 'mse_decode_vs_true': mse_decode_vs_true,
                # 'mean_H_difference': mean_H_diff,
                # 'std_H_difference': std_H_diff
                # }

                # csv_dict_all_data.append(csv_dict)

                # # Call the save_csv_data function with the dictionary
                # save_data_to_csv(csv_dict, csv_save_path, is_list=False)
                # save_data_to_csv(csv_dict_all_data, compile_path, is_list=True) 

                data_dict = {
                    'neural_data': all_neural_data,
                    'sessions_metadata': all_sessions_data,
                    'embeddings_3d': all_embeddings_3d,
                    'principal_curves_3d': all_principal_curves_3d,
                    'curve_params_3d': all_curve_params_3d,
                    'binned_hipp_angle': all_binned_hipp_angle,
                    'binned_true_angle': all_binned_true_angle,
                    'binned_est_gain': all_binned_est_gain,
                    'binned_high_vel': all_binned_high_vel,
                    'decoded_angles': all_decoded_angles,
                    'filtered_decoded_angles_unwrap': all_filtered_decoded_angles_unwrap,
                    'decode_H': all_decode_H,
                    'session_idx': all_session_idx,
                    'rat': all_rat,
                    'day': all_day,
                    'epoch': all_epoch,
                    'num_skipped_clusters': all_num_skipped_clusters,
                    'num_used_clusters': all_num_used_clusters,
                    'avg_skipped_cluster_isolation_quality': all_avg_skipped_cluster_isolation_quality,
                    'avg_used_cluster_isolation_quality': all_avg_used_cluster_isolation_quality,
                    'mean_distance_to_principal_curve': all_mean_distance_to_principal_curve,
                    'mean_angle_difference': all_mean_angle_difference,
                    'shuffled_mean_angle_difference': all_shuffled_mean_angle_difference,
                    'SI_score_hipp': all_SI_score_hipp,
                    'SI_score_true': all_SI_score_true,
                    'mse_decode_vs_true': all_mse_decode_vs_true,
                    'mean_H_difference': all_mean_H_difference,
                    'std_H_difference': all_std_H_difference
                }


                # Save all data to same .mat file
                mat_filename = os.path.join(base_path, f'{save_folder}_all_sessions_data.mat')
                savemat(mat_filename, data_dict)

                pdf.close()

    

            

Analysis of trials

In [ ]:

plot_scatter(
    data=data,
    x_key="num_clusters",
    y_key="data_quality_score",
    x_label="Number of Clusters",
    y_label="Data Quality Score",
    title="Number of Clusters vs Data Quality Score"
)